In [2]:
import torch
import torch.nn as nn
import os
from cs336_basics.myModule import toy_Transformer_lm

In [6]:
test_ml = toy_Transformer_lm(10,10,20,2,4,40,0.00001)
for name, p in test_ml.named_parameters():
    print(name, p.shape, p.requires_grad)

tk_embd.embd torch.Size([10, 20]) True
blocks.0.norm1.gain torch.Size([20]) True
blocks.0.atte.c_attention.weight torch.Size([60, 20]) True
blocks.0.atte.proj.weight torch.Size([20, 20]) True
blocks.0.norm2.gain torch.Size([20]) True
blocks.0.ff.W1 torch.Size([40, 20]) True
blocks.0.ff.W2 torch.Size([20, 40]) True
blocks.0.ff.W3 torch.Size([40, 20]) True
blocks.1.norm1.gain torch.Size([20]) True
blocks.1.atte.c_attention.weight torch.Size([60, 20]) True
blocks.1.atte.proj.weight torch.Size([20, 20]) True
blocks.1.norm2.gain torch.Size([20]) True
blocks.1.ff.W1 torch.Size([40, 20]) True
blocks.1.ff.W2 torch.Size([20, 40]) True
blocks.1.ff.W3 torch.Size([40, 20]) True
norm.gain torch.Size([20]) True
out_embd.weight torch.Size([10, 20]) True


In [ ]:

class toy_AdamW(torch.optim.Optimizer):
    def __init__(self, params,betas=(0.9,0.999),weight_decay = 0,eps = 1e-8,lr = 1e-3) -> None:
        defaults = dict(lr=lr,beta1 = betas[0],beta2 = betas[1], decay = weight_decay, eps = eps)
        super().__init__(params, defaults)
        
    @torch.no_grad()
    def step(self, closure = None):
        loss = closure() if closure is not None else None
        for group in self.param_groups:
            lr = group["lr"]
            beta1 = group["beta1"]
            beta2 = group["beta2"]
            decay = group["decay"]
            eps = group["eps"]
            
            for p in group["params"]:
                if p.grad is None:
                    continue
                state = self.state[p] # Get state associated with p.
                t = state.get("t", 0) + 1 # Get iteration number from the state, or initial value.
                m = state.get("m", torch.zeros_like(p))
                v = state.get("v", torch.zeros_like(p))
                grad = p.grad
                
                m_new = beta1 * m + (1 - beta1)* grad
                v_new = beta2 * v + (1 - beta2)* grad**2
                lr_now = lr * ((1 - beta2**t)**(0.5)) / (1 - beta1**t)
                p -= lr_now * m_new / (v_new**0.5 + eps) + lr*decay*p
                
                state["t"] = t
                state["m"] = m_new
                state["v"] = v_new                
        return loss
        

In [ ]:
@torch.no_grad()
def toy_grad_clip(parameters,max_l2_norm):
    norm_all = torch.zeros(1)
    for p in parameters:
        if p.grad is not None:
            norm_all += p.grad.norm().square()
    norm_all = norm_all.sqrt()
    if norm_all > max_l2_norm:
        scale = max_l2_norm / norm_all
        for p in parameters:
            if p.grad is not None:
                p.grad.mul_(scale)
    return

In [ ]:
a = nn.Parameter(torch.tensor([[1,2,3],[4,5,6]],dtype=torch.float32))


tensor(91.0000, grad_fn=<PowBackward0>)